In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import xarray as xr
import calendar
import os

In [ ]:
ameriflux_df = pd.read_csv("ameriflux.sites.north_hem.nl5yr.location.csv",
                           index_col=0)

In [ ]:
xcel_input = pd.ExcelFile("/glade/work/yifanc/proj/wpo2023/scm/forcing/ERA5_variable.SCM.input.xlsx")
var_df = xcel_input.parse(sheet_name='era5_var')
oroot_raw_dir = "/glade/derecho/scratch/yifanc/wpo2023/SCM/raw/"
oroot_process_dir = "/glade/derecho/scratch/yifanc/wpo2023/SCM/processed/"

In [7]:
var_df

,Variable,prefix,code_table,param_code,var_abbr,postfix,agg_method,agg_time
0,geopotential,e5.oper.an.pl,128,129,z,ll025sc,daily,time
1,specific_humidity,e5.oper.an.pl,128,133,q,ll025sc,daily,time
2,temperature,e5.oper.an.pl,128,130,t,ll025sc,daily,time
3,u_component_of_wind,e5.oper.an.pl,128,131,u,ll025uv,daily,time
4,v_component_of_wind,e5.oper.an.pl,128,132,v,ll025uv,daily,time
5,vertical_velocity,e5.oper.an.pl,128,135,w,ll025sc,daily,time
6,surface_pressure,e5.oper.an.sfc,128,134,sp,ll025sc,monthly,time
7,skin_temperature,e5.oper.an.sfc,128,235,skt,ll025sc,monthly,time
8,2m_temperature,e5.oper.an.sfc,128,167,2t,ll025sc,monthly,time
9,10m_u_component_of_wind,e5.oper.an.sfc,128,165,10u,ll025sc,monthly,time


In [8]:
site = 'US-Whs'
site_lat = ameriflux_df['Latitude (degrees)'].loc[site]
site_lon = ameriflux_df['Longitude (degrees)'].loc[site]
osite_raw_dir = oroot_raw_dir + "%s/"%site

In [9]:
def convert_rad_data(da):
    initial_time_expanded = np.repeat(da["forecast_initial_time"],len(da['forecast_hour']))
    time_offsets  = pd.to_timedelta(np.tile(da["forecast_hour"],len(da["forecast_initial_time"])),unit='h')
    new_time_dim = initial_time_expanded + time_offsets
    stacked = da.stack(time=("forecast_initial_time", "forecast_hour"))
    stacked = stacked.drop_vars(['time', 'forecast_initial_time', 'forecast_hour'])
    stacked = stacked.assign_coords(time=("time", new_time_dim.values))
    stacked = stacked.transpose("time", "latitude", "longitude")
    return stacked


In [12]:
def previous_month_year(month: int, year: int) -> tuple[int, int]:
    if month == 1:
        return 12, year - 1
    else:
        return month - 1, year

def starts_with_number(s: str) -> bool:
    return s[0].isdigit() if s else False

def get_days_in_month(year, month):
  if not (1 <= month <= 12):
    raise ValueError("Month must be between 1 and 12.")

  # monthrange returns a tuple: (weekday of first day, number of days)
  # We are interested in the second element (number of days)
  _, num_days = calendar.monthrange(year, month)
  return num_days

In [13]:
out_varname_dic = {}
for row in var_df.iterrows():
    var_abbr = row[1]['var_abbr']
    if var_abbr == '2t':
        var_out = 't2m'
    else:
        var_out = var_abbr
    out_varname_dic.update({var_abbr:var_out})

In [14]:
osite_process_dir = oroot_process_dir + "%s/"%site
if not os.path.exists(osite_process_dir):
    os.mkdir(osite_process_dir)

In [ ]:
start_year = 2019
end_year = 2025
for year in range(start_year,end_year):
    ystr = str(year)
    year_pre = year - 1
    ystr_pre = str(year_pre)
    agg_file = osite_process_dir + \
               "processed_era5.%s.%s.nc"%(ystr,site)
    da_list = {}
    for row in var_df.iterrows():
        variable = row[1]['Variable']
        prefix = row[1]['prefix']
        code_table = row[1]['code_table']
        param_code = "%03i"%row[1]['param_code']
        var_abbr = row[1]['var_abbr']
        postfix = row[1]['postfix']
        agg_time = row[1]['agg_time']
        print(variable,prefix,code_table,param_code,var_abbr,postfix)
        ofile_month  = osite_raw_dir + "%s.%i_%s_%s.%s.%s.%s.nc"%(prefix,code_table,param_code,var_abbr,postfix,site,ystr)
        ofile_premonth = osite_raw_dir + "%s.%i_%s_%s.%s.%s.%s.nc"%(prefix,code_table,param_code,var_abbr,postfix,site,ystr_pre)
        
        ds = xr.open_dataset(ofile_month)
        if starts_with_number(var_abbr):
            var_name = "VAR_%s"%(var_abbr.upper())
        else:
            var_name = var_abbr.upper()
        da = ds[var_name]
        if prefix == "e5.oper.fc.sfc.accumu":
            da = convert_rad_data(da)
            if year != start_year:
                ds_premonth = xr.open_dataset(ofile_premonth)
                da_premonth = ds_premonth[var_name]
                da_premonth = convert_rad_data(da_premonth)
                da = xr.concat([da_premonth,da],dim='time')
            da = da.sortby('time', ascending=True)
            da = da.sel(time=slice("%s-01-01 00"%(ystr),"%s-12-31 23"%(ystr)))
        else:
            da = da.sortby('time', ascending=True)
        print(da.time.values[0],da.time.values[-1])
        da_list.update({out_varname_dic[var_abbr]:da})
    ds_out = xr.Dataset(da_list)
    ds_out = ds_out.rename({'time':'valid_time','level':'pressure_level'})
    ds_out.to_netcdf(agg_file,format="NETCDF4_CLASSIC")

geopotential e5.oper.an.pl 128 129 z ll025sc
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
specific_humidity e5.oper.an.pl 128 133 q ll025sc
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
temperature e5.oper.an.pl 128 130 t ll025sc
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
u_component_of_wind e5.oper.an.pl 128 131 u ll025uv
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
v_component_of_wind e5.oper.an.pl 128 132 v ll025uv
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
vertical_velocity e5.oper.an.pl 128 135 w ll025sc
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
surface_pressure e5.oper.an.sfc 128 134 sp ll025sc
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
skin_temperature e5.oper.an.sfc 128 235 skt ll025sc
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
2m_temperature e5.oper.an.sfc 128 167 2t ll025sc
2019-01-01T00:00:00.000000000 2019-12-31T23:00:00.000000000
10m_u_component